In [1]:
import pandas as pd
import numpy as np
# import pymongo
# import requests
# from bs4 import BeautifulSoup
import json
# from selenium import webdriver
# import sys
# import time
# import re

#### import basic artist list csv as a dataframe & also create artist list

In [2]:
def decode_text(list_name):
    a = []
    for x in list_name:
        a.append(x.decode('utf-8'))
    return a

In [3]:
with open('data/artist_df.csv') as f:
    artist_df = pd.read_csv(f)
artist_names = list(artist_df['artist_name'])
artist_names = decode_text(artist_names)
artist_df.tail()

,artist_id,image_url,artist_name,site_url
8544,8544,/assets/big-default-user-pic.png?v=1483729361,Éric Poncet,/pros/eric-poncet
8545,8545,/assets/big-default-user-pic.png?v=1447212641,Óscar Silva,/pros/oscar-silva
8546,8546,/assets/big-default-user-pic.png?v=1433288081,Øystein Brun,/pros/oystein-brun
8547,8547,/assets/big-default-user-pic.png?v=1433288081,Øystein Landsverk,/pros/oystein-landsverk
8548,8548,/assets/big-default-user-pic.png?v=1445558142,Þráinn Árni Baldvinsson,/pros/thrainn-arni-baldvinsson


#### import full artist and equipment data files

In [4]:
with open('data/cleaned_pedal_data.csv') as f:
    pedal_data = pd.read_csv(f)
with open('data/artist_data.json') as f:
    artist_data = json.load(f)

In [5]:
clean_pedal_data = pedal_data.drop('equipment_name',axis=1)
clean_pedal_data = clean_pedal_data.drop_duplicates()
clean_pedal_data.reset_index(drop=True,inplace=True)
clean_pedal_data.head()

,cleaned_name,brand_name,product_name,product_category,instrument_type,equipment_urls
0,2 Note Torpedo VB-101,2 Note,Torpedo VB-101,Speaker Simulator,Guitar,/items/2-note-torpedo-vb-101
1,3Leaf Audio Proton Envelope Filter,3Leaf,Audio Proton Envelope Filter,Filter,Guitar,/items/3leaf-audio-proton-envelope-filter-guit...
2,4114 Custom Effects Flux Capacitor Delay,4114 Custom Effects,Flux Capacitor Delay,Delay,Guitar,/items/4114-custom-effects-flux-capacitor-dela...
3,A/DA Flanger,A/DA,Flanger,Flanger,Guitar,/items/a-da-flanger
4,Abbott's Music Handwired Fuzz,Abbott's Music,Handwired Fuzz,Fuzz,Guitar,/items/abbott-s-music-handwired-fuzz-pedal


In [6]:
artist_data['Ryan Adams']

{u'genres': [u'Rock', u'Folk, World, & Country'],
 u'member of': [u'Phil Lesh and Friends',
  u'Whiskeytown',
  u'Ryan Adams & The Cardinals'],
 u'pedals': [u'Electro-Harmonix Holy Grail Reverb Guitar Effects Pedal',
  u'Boss CH-1 SUPER Chorus Pedal',
  u'Xotic Effects EP Booster Pedal',
  u'Boss DD-7 Digital Delay',
  u'4114 Custom Effects - Flux Capacitor Delay Pedal',
  u'Boss DM-2 Analog Delay',
  u'Boss DC-2 Dimension C',
  u'Boss CE-1 Chorus Ensemble Pedal',
  u'DOD Stereo Chorus 565 Performer',
  u'BOSS AP-2 Phase II',
  u'Boss CE-3 Chorus',
  u'Electro-Harmonix Big Muff Pi',
  u'DOD FX90 Delay',
  u'DOD FX65 Stereo Chorus',
  u'Maestro Fuzz-Tone FZ-1S',
  u'Dynastar M-31 Vacuum Tube Overdrive Unit',
  u'Fuzzrocious Afterlife',
  u'Fuzzrocious Pedals The Demon',
  u'JHS + PaxAm, Ryan Adams VCR',
  u'Old Blood Noise Endeavours Black Fountain Delay',
  u'Chase Bliss Audio Tonal Recall Analog Delay'],
 u'similar artists': [u'Whiskeytown',
  u'Jayhawks',
  u"Old 97's",
  u'Drive-By 

#### clean and create lists of similar artists, pedals, and genres

In [154]:
fix = [u'Blues',
 u'Brass & Military',
 u"Children's",
 u'Classical',
 u'Electronic & DJ',
 u'Folk, World, & Country',
 u'Funk / Soul',
 u'Hip Hop',
 u'Jazz',
 u'Latin',
 u'Non-Music',
 u'Orchestra',
 u'Pop',
 u'Reggae',
 u'Rock',
 u'Stage & Screen',
 u'Unknown']

genres = []
similars = []
for key1,value2 in artist_data.iteritems():
    for key2,value2 in value2.iteritems():
#         if fix_name in value2 and key2 == 'genres':
#             fix.append(key1)
        for x in value2:
            if key2 == 'genres' and x in fix:
                genres.append(x)
            elif key2 == 'similar artists' or key2 == 'member of':
                similars.append(x)
genres = list(set(genres))
similars = list(set(similars))
pedals = list(set(clean_pedal_data['cleaned_name']))

#### import ratings df for spark ALS

In [152]:
with open('data/artist_style.csv') as f:
    artist_style_df = pd.read_csv(f)
artist_style_df.head()

,user,item,rating
0,"""Captain"" Kirk Douglas",Rock,1
1,"""Captain"" Kirk Douglas",Funk / Soul,1
2,"""Captain"" Kirk Douglas",The Roots,1
3,"""J Dog"" Jorel Decker",Hip Hop,1
4,"""J Dog"" Jorel Decker",Rock,1


In [146]:
user_style_df[user_style_df['user']==1222]

,user,item,rating
4612,1222,44,1
4613,1222,2565,1
4614,1222,5169,1
4615,1222,8718,1
4616,1222,4894,1
4617,1222,8138,1
4618,1222,3172,1


In [9]:
with open('data/artist_owned.csv') as f:
    artist_owned_df = pd.read_csv(f)
artist_owned_df.head()

,user,item,rating
0,"""Captain"" Kirk Douglas",MI Audio Crunch Box,1
1,"""Captain"" Kirk Douglas",Maxon Phase Tone,1
2,"""Captain"" Kirk Douglas",Ibanez TS9 Tube Screamer,1
3,"""Captain"" Kirk Douglas",Boss TU-2 Chromatic Tuner,1
4,"""Captain"" Kirk Douglas",Durham Electronics Sex Drive,1


#### create user_id & pedal_id

In [159]:
def add_ids(input_list):
    id_list = []
    for i,a in enumerate(input_list):
        id_list.append([i,a])
    return id_list

def get_ids(ids,df):
    for num,user in ids:         
        df = df.replace(user,num)
    return df

In [161]:
user_ids = add_ids(artist_names)
item_ids = add_ids(pedals + genres + similars)
# only have to do below once then save to import later
# user_style_df = get_ids(user_ids,artist_style_df)
# user_style_df = get_ids(item_ids,user_style_df)
# user_item_df = get_ids(user_ids,artist_owned_df)
# user_item_df = get_ids(item_ids,user_item_df)
# user_style_df.drop_duplicates(inplace=True)
# user_item_df.drop_duplicates(inplace=True)
# with open('data/user_style_df.csv','w') as f:
#     user_style_df.to_csv(f,index=False)
# with open('data/user_item_df.csv','w') as f:
#     user_item_df.to_csv(f,index=False)

#### import now created and saved user ratings files

In [512]:
with open('data/user_style_df.csv') as f:
    user_style_df = pd.read_csv(f)
with open('data/user_item_df.csv') as f:
    user_item_df = pd.read_csv(f)

In [211]:
user_style_df = user_style_df.astype(int)
user_style_df.head()

,user,item,rating
0,0,2550,1
1,0,2551,1
2,0,6553,1
3,1,2548,1
4,1,2550,1


In [513]:
user_item_df = user_item_df.astype(int)
user_item_df.head()

,user,item,rating
0,0,296,1
1,0,1872,1
2,0,105,1
3,0,1439,1
4,0,2331,1


In [314]:
#reshape data into matrix for cosine similarity calc
user_style_pivot = user_style_df.pivot(index='user',columns='item')
user_style_pivot = user_style_pivot.fillna(0)
user_style_pivot.columns = user_style_pivot.columns.droplevel(0)
# user_style_matrix = user_style_pivot.as_matrix()
# user_style_matrix = user_style_matrix

#update dataframe to include 0s for items they do not own 
user_item_pivot = user_item_df.pivot(index='user',columns='item')
user_item_pivot = user_item_pivot.fillna(0)
user_item_pivot.columns = user_item_pivot.columns.droplevel(0)
user_item_pivot['user'] = user_item_pivot.index
full_user_item_df = pd.melt(user_item_pivot, id_vars='user', value_vars=list(user_item_pivot.columns[:2500]),
            var_name='item', value_name='rating')

In [350]:
user_style_pivot.head()

item,2538,2539,2540,2541,2542,2543,2544,2545,2546,2547,...,9569,9570,9571,9572,9573,9574,9575,9576,9578,9579
user,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [315]:
user_item_df.head()

,user,item,rating
0,0,0,0.0
1,1,0,0.0
2,4,0,0.0
3,6,0,0.0
4,8,0,0.0


#### create cosine similarity matrix

In [405]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

def get_similarity_matrix(user_matrix): 
    user_matrix_sparse = sparse.csr_matrix(user_matrix)
    similarities = cosine_similarity(user_matrix_sparse)
    return similarities

In [406]:
similarity_matrix = get_similarity_matrix(user_style_matrix)

In [464]:
top = np.argsort(similarity_matrix[0])[::-1][:6]
top

array([   0, 4564, 5145, 2123, 6874, 2124])

In [454]:
def generate_new_user(sim_matrix):
    new_user = np.random.randint(2, size=(1,sim_matrix.shape[1]))
    return new_user

In [485]:
def get_similar_users(new_user,existing_users,num_users=5):
    new_matrix = np.append(new_user,existing_users,axis=0)
    sim_matrix = get_similarity_matrix(new_matrix)
    top_similar_users = np.argsort(sim_matrix[0])[::-1][1:num_users+1]
    top_similar_users
    return top_similar_users

In [486]:
new_user = generate_new_user(user_style_matrix)
top_n_similar_users = get_similar_users(new_user,user_style_matrix,num_users=5)
top_n_similar_users

array([5264, 5631, 7410, 3240,  532])

#### start spark to create sql tables

In [523]:
import pyspark
from pyspark.sql.types import *
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating

spark = pyspark.sql.SparkSession.builder \
            .master("local") \
            .appName("Don't Fret") \
            .getOrCreate()
sc = spark.sparkContext
spark, sc

(<pyspark.sql.session.SparkSession at 0x11847c790>,
 <pyspark.context.SparkContext at 0x1182915d0>)

#### train spark ALS model

In [524]:
user_item_spark_df = spark.createDataFrame(user_item_df)
user_item_rdd = user_item_spark_df.rdd
train, test = user_item_rdd.randomSplit([0.8, 0.2], seed=2711)
testdata = test.map(lambda p: (p[0], p[1]))

In [525]:
user_item_spark_df.describe().show()

KeyboardInterrupt: 

In [526]:
model = ALS.trainImplicit(train, rank=5, iterations=5, nonnegative=True)

KeyboardInterrupt: 

#### MSE for test data

In [505]:
# predictions = model.predictAll(testdata).map(lambda r: ((r[0], r[1]), r[2]))
# ratesAndPreds = testdata.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
# MSE = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean()
# print("Mean Squared Error = " + str(MSE))

In [527]:
#pickle model
import cPickle as pickle
with open("data/model.pkl", 'w') as f:
    pickle.dump(model, f)

TypeError: can't pickle lock objects

In [510]:
model.predict(1,1)

KeyboardInterrupt: 

#### generate recommendations for new user defined above

In [ ]:
#unpickle model
with open("data/model.pkl") as f_un:
    model_unpickled = pickle.load(f_un)

In [ ]:
def get_user_id(user_name):
    for user_id,user in user_ids:
        if user == user_name:
            return user_id

def top_n_recommendations(user_id,n):
    recommends = []
    for user,item,pred in model.recommendProducts(user_id,n):
         for item_id,item_name in item_ids:
                if item == item_id:
                    recommends.append([item_name,pred])
    return recommends

In [ ]:
def top_n_recommendations(user_id):
    for user,item,pred in model.recommendProducts(user_id,n):
         for item_id,item_name in item_ids:
                if item == item_id:
                    print item_name
def get_recommendations(sim_users,n):
    user_recommendations = []
    for user in sum_users:
        recs = top_n_recommendations(user,n)
        for rec,pred in recs:
            user_recommendations.append([rec,pred])
    return user_recommendations